<img src='https://www.met.no/om-oss/logo/_/image/73f29cde-219f-487b-809c-9cdd61032c78:2efc46ce776f5f5337c4b0156ae0cbaa3b6bf6fe/width-768/Met_RGB_Horisontal.jpg' width=200 align=right>
<img src='https://raw.githubusercontent.com/norkyst/norkyst-logo/refs/heads/main/png/horizontal_35_91_100.png' width=200 align=right>

# __UNDER CONSTRUCTION__
# Freshwater height and potential energy

This notebook will give instructions on how to find freshwater height and potential energy using Norkyst v3 data.

In [2]:
import numpy as np
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
import cmocean.cm as cmo
import xarray as xr

In [4]:
path = "https://thredds.met.no/thredds/dodsC/fou-hi/norkystv3_800m_m00_be"

ds = xr.open_dataset(path)

In [ ]:
ds

In [3]:
def fwh(salt, z_w, saltref=35.0, maxdepth=-10.0):

    """ 
    This function returns the freshwater height, which is defined as
    the integral from a given depth to the surface, using the integrand
        
        max(S_ref - S,0)/S_ref
                
    where S_ref is a reference salinity value, typically chosen as a value
    representative of open ocean conditions where there is little influence
    from riverine forcing. The freshwater height can be interpreted as 
    the volume of freshwater per m^2 that must be added to a water mass 
    with salinity S_ref to obtain the observed or modeled salinity profile.

    For example, if S_ref = 35.0, and the freshwater height between z=(-10,0)
    is 5 m, then the total amount of salt is equivalent to having S = 35.0 
    in the bottom 5 m, and S = 0.0 in the top 5 m. In reduced gravity models, 
    the potential energy anomaly can be shown to be proportional to the geostrophic 
    surface current stream-function, see Gustafsson (Cont. Shelf Res., 19(8), 1999).  
    
    Keep in mind that maps based on the output from this function might appear
    strange if the chosen maximum depth is larger than the minimum depth of
    the model, hence it is best to mask the map where the local depth is smaller
    than the maximum depth.
    
    2025-04-15, kaihc@met.no
    
    Usage:
    
        fresh_water_height = fwh(salt, z_w, saltref=35.0, maxdepth=-10.0)
        
    Variables:

        fresh_water_height  - 3D freshwater height in [m] (ndarray [T,Y,X])
        salt                - 4D salinity field from ROMS (ndarray [T,Z,Y,X])
        z_w                 - 4D depth values of w-points (ndarray [T,Z,Y,X])
        saltref             - reference salinity value, default = 35.0
        maxdepth            - thickness of layer, default = -10.0
                              time dependent zeta is taken into account
    """

    # Truncate z vector, keeping in mind that the surface coordinate is time dependent
    z = np.where(z_w < maxdepth + z_w[:,-1:,:,:], maxdepth + z_w[:,-1:,:,:], z_w)

    # Calculate dz
    dz = np.diff(z, axis=1)

    # Calculate integrand
    S = np.max(saltref-salt,0)/saltref

    # Integrate and return
    return np.sum(S*dz, axis=1)

In [7]:
ds

<xarray.Dataset>
Dimensions:                  (X: 2747, Y: 1148, depth: 15, time: 11591)
Coordinates:
  * X                        (X) int32 0 800 1600 ... 2195200 2196000 2196800
  * Y                        (Y) int32 0 800 1600 2400 ... 916000 916800 917600
  * depth                    (depth) float64 0.0 1.0 2.0 ... 100.0 200.0 300.0
  * time                     (time) datetime64[ns] 2024-01-01 ... 2025-05-03
    lon                      (Y, X) float64 ...
    lat                      (Y, X) float64 ...
Data variables:
    projection_stere         int32 ...
    forecast_reference_time  float64 ...
    h                        (Y, X) float64 ...
    zeta                     (time, Y, X) float32 ...
    u_eastward               (time, depth, Y, X) float32 ...
    v_northward              (time, depth, Y, X) float32 ...
    w                        (time, depth, Y, X) float32 ...
    temperature              (time, depth, Y, X) float32 ...
    salinity                 (time, depth, Y, X) float32 ...
    AKs                      (time, depth, Y, X) float32 ...
    Uwind_eastward           (time, Y, X) float32 ...
    Vwind_northward          (time, Y, X) float32 ...
Attributes: (12/37)
    id:                         0413e08c-cf07-4a2f-a7a0-3d25aea39e6d
    naming_authority:           no.met
    operational_status:         Operational
    iso_topic_category:         oceans
    activity_type:              Numerical Simulation
    keywords_vocabulary:        GCMDSK:GCMD Science Keywords:https://gcmd.ear...
    ...                         ...
    dataset_production_status:  In Work
    license:                    https://spdx.org/licenses/CC-BY-4.0 (CC-BY-4.0)
    title:                      Norkyst-800m - ROMS, Norkyst-800m 120 hours o...
    summary:                    Norkyst-800m (Norwegian Coast 800m horizontal...
    title_no:                   120 timers prognoser fra havmodellen Norkyst-...
    summary_no:                 NorKyst-800m (Norske kystområder med 800m hor...

In [5]:
salt = ds.salinity
z_w = ds.depth

In [6]:
fwh(salt, z_w)

IndexError: too many indices